# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

In [2]:
import gym
import random
import torch
from torch import nn
import numpy as np
from collections import deque
import matplotlib.pyplot as5plt
%matplotlib inline



In [25]:

a = torch.randn(50, 100, 1)  
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device that gonna be used",device)

print(a.size())
m = nn.Conv1d(100, 100, kernel_size = 8 , stride = 1 , padding = 5 ) 
out = m(a)
print("out size",out.size())
print("covolutional layer",m)

cpu
torch.Size([50, 100, 1])
out size torch.Size([50, 100, 4])
covolutional layer Conv1d(100, 100, kernel_size=(8,), stride=(1,), padding=(5,))


In [3]:
import os
path = r"C:\Users\adith\Desktop\Meng Robotics\reinforcement\Banana\deep-reinforcement-learning\p1_navigation\Banana_Windows_x86_64\Banana.exe"


In [4]:
from unityagents import UnityEnvironment
import numpy as n

The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [5]:
# please do not modify the line below
env = UnityEnvironment(file_name=path)
# env = UnityEnvironment(file_name="C:\Reinforcement\Banana\deep-reinforcement-learning\p1_navigation\Banana_Windows_x86_64\Banana_Windows_x86_64\Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [6]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [7]:
print(brain_name)

BananaBrain


### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [8]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)
State_type = type(state)
print("state_type", State_type)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37
state_type <class 'numpy.ndarray'>


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [9]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    #print(len(next_state))
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


In [10]:
!jupyter nbextension enable varInspector/main

Enabling notebook extension varInspector/main...
      - Validating: ok


In [11]:
from dqn_agent import Agent
agent = Agent(state_size, action_size,State_type ,seed=0)

Device which is used is  cuda:0


In [12]:
print(brain_name , brain_name)

BananaBrain BananaBrain


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

a = torch.tensor([1.0,1,1] , requires_grad=False)
b = torch.tensor([2.0,2,2] , requires_grad=True)
print(b.size())
b = a*b

loss = F.mse_loss(a,b, reduce = False)
print(loss.size())
loss = torch.mean(loss)
print(loss)

torch.Size([3])
torch.Size([3])
tensor(1., grad_fn=<MeanBackward1>)


E:\Anaconda\lib\site-packages\torch\nn\_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [14]:


def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        #print("hello")
        for t in range(max_t):
            action = agent.act(state, eps)
            action = action.astype(np.int32)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            agent.step(state, action, reward, next_state, done)
            #print(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 1 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=16.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            if np.mean(scores_window)>=16.0:
                break
    return scores

scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 1	Average Score: 1.00
Episode 2	Average Score: 0.00
Episode 3	Average Score: 0.00
Episode 4	Average Score: 0.25
Episode 5	Average Score: -0.40
Episode 6	Average Score: -0.17
Episode 7	Average Score: -0.14
Episode 8	Average Score: -0.12
Episode 9	Average Score: -0.11
Episode 10	Average Score: -0.20
Episode 11	Average Score: -0.27
Episode 12	Average Score: -0.25
Episode 13	Average Score: -0.23
Episode 14	Average Score: -0.07
Episode 15	Average Score: -0.13
Episode 16	Average Score: 0.00
Episode 17	Average Score: -0.12
Episode 18	Average Score: -0.11
Episode 19	Average Score: -0.05
Episode 20	Average Score: -0.05
Episode 21	Average Score: -0.05
Episode 22	Average Score: -0.05
Episode 23	Average Score: -0.09
Episode 24	Average Score: 0.00
Episode 25	Average Score: 0.00
Episode 26	Average Score: 0.00
Episode 27	Average Score: -0.04
Episode 28	Average Score: 0.07
Episode 29	Average Score: 0.07
Episode 30	Average Score: 0.03
Episode 31	Average Score: 0.06
Episode 32	Average Score: 0.0

Episode 260	Average Score: 5.71
Episode 261	Average Score: 5.76
Episode 262	Average Score: 5.84
Episode 263	Average Score: 5.89
Episode 264	Average Score: 5.97
Episode 265	Average Score: 5.94
Episode 266	Average Score: 5.96
Episode 267	Average Score: 5.98
Episode 268	Average Score: 6.05
Episode 269	Average Score: 6.03
Episode 270	Average Score: 6.02
Episode 271	Average Score: 6.09
Episode 272	Average Score: 6.16
Episode 273	Average Score: 6.20
Episode 274	Average Score: 6.23
Episode 275	Average Score: 6.26
Episode 276	Average Score: 6.29
Episode 277	Average Score: 6.26
Episode 278	Average Score: 6.33
Episode 279	Average Score: 6.40
Episode 280	Average Score: 6.44
Episode 281	Average Score: 6.48
Episode 282	Average Score: 6.48
Episode 283	Average Score: 6.56
Episode 284	Average Score: 6.67
Episode 285	Average Score: 6.72
Episode 286	Average Score: 6.78
Episode 287	Average Score: 6.85
Episode 288	Average Score: 6.91
Episode 289	Average Score: 6.90
Episode 290	Average Score: 6.91
Episode 

KeyboardInterrupt: 

In [ ]:
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))


In [ ]:
agent.qnetwork_local

In [ ]:

env_info = env.reset(train_mode=False)[brain_name]

In [ ]:
# load the weights from file
# env = UnityEnvironment(file_name=path)
env_info = env.reset(train_mode=False)[brain_name]
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
state = env_info.vector_observations[0]
score =0
for i in range(300):
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations[0]
    for j in range(200):
        action = agent.act(state)
        action = np.int32(action)
        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        done = env_info.local_done[0]
        state = next_state
        score += reward
        if done:
            break
            print(score)
            

When finished, you can close the environment.

In [ ]:
env.close()

In [ ]:
from collections import namedtuple, deque
from queue import PriorityQueue
import numpy as np

Pri = PriorityQueue()

Pri.put((0.1,(1,2)))
Pri.put((2,(2,2)))
Pri.put((3,(9,2)))

experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
Type = [('s0', 'float'),('s1', 'float'),('s2', 'float'),('s3', 'float')]

states = np.array((1,1,1,1))
st = type(states)
print(st)
print("st", states , states.dtype)
a  =(states,1,1,1,True)
print("a ", a)
probs = [0.34 ,0.33 ,0.33]
print(len(probs))
indexarray = np.arange(len(probs))
print(indexarray)

memory = deque()
memory.append((5,a))
memory.append((7,a))
memory.append((2,a))
m = [*zip(*memory)]
print("m",m)
mu = np.array(m[0])
expType = [('state', st),('action','float'),('reward','float'), ('next_state','float'), ('done','bool')]
mup = np.array([i for i in m[1]] , dtype = expType )
mup = np.array([], dtype = expType)
print(mup)
x = np.array([a] , dtype = expType)
mup = np.append(mup,x)
mup = np.append(mup,x)
print("state",mup['done'])
print("mu", mup)
print("size",np.size(mup))
mup['action'][[0,1]] = [0,5]
print(mup)
mup = np.delete(mup , 1)
print("size",np.size(mup))
print("state", mup['state'])

# ind = np.array([0,1])
# print("indezed", mup[ind])
# pu = m[1]
# su = np.sum(mu)
# ar = [*zip(mu,pu)]
# print(ar)

# zipped = np.array(ar,object)
# #zipped = zipped.flatten()
# print(zipped)

# ch = np.random.choice(indexarray , 2 , p=probs)
# final = zipped[ch]
# print(final)
# print("zip",zipped, ch)
# print(m[0], su, pu)
# print(*zip(*memory))
# print(memory)
# p , x = memory[1]
# print(p , x)
# print(memory[1])
# del memory[2]
# print(final[0])
# a , b = final[0]
# print(a , b)

In [ ]:
import heapq
x =[(1,0) , (2,2) , (3,5)]
y =[]
x.append(-9)
for 
    heapq.heappush(y , i)

u = [heapq.heappop(y) for i in range(len(y))]   
print(u)


In [ ]:
e = experience(1,1,1,1,True)
memg = deque()
memg.append(e)
memg.append(e)
memg.append(e)
print(np.vstack([e.state for e in memg if e is not None]))
print(np.vstack(mup['state']))
c = [0,1]
p = mup['state'][c]
print(p)

In [ ]:
from queue import PriorityQueue

Pri = PriorityQueue()

Pri.put((0.1,(1,2)))
Pri.put((2,(2,2)))
Pri.put((3,(9,2)))

experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
a  = experience(1,1,1,1,True)
b = (1,a)
print(b)
b[0]
Pri.put(b)
Pri.put(b)
Pri.put(b)
print(Pri.qsize())
a = Pri.get()
print(a)
print(Pri.qsize())
print(Pri[2])

In [ ]:
print(Pri)


### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 

In [ ]:
import numpy as np
s= [1,2,3,4]
P=[0.25,0.25,0.25,0.25]
c = np.random.choice(s,2,p=P)
print(c)

In [ ]:
a = [1,2]

In [ ]:
p = np.array(a)
z = p ** 2
z